In [195]:
from math import sqrt
from numpy import std
from sympy import Float, init_printing, Array, relational, symbols
init_printing()

### Input data

In [196]:
# Input data for Hook-Jeeves' method task

hook_jeeves_accuracy = 0.01
hook_jeeves_step_increment = 2
hook_jeeves_x_start_vector = [8, 6]
hook_jeeves_delta_x_start_vector = [0.6, 0.8]


def hook_jeeves_target_function(x1, x2):
    return (x1 - 4)**2 +2*x2**2


# Input data for Nelder-Mead's method task

nelder_mead_start = [[6, 5], [8, 7], [6, 7]]
nelder_mead_accuracy = 0.001


def nelder_mead_target_function(x1, x2):
    return 2*(x1 - 3)**2 + (x2 - 4)**2


### Hook-Jeeves's methods


In [197]:
def vector_abs(vector):
    result= 0
    for v in vector:
        result += v**2
    return sqrt(result)


def increment_vector(vector, index, increment):
    new_vector = vector.copy()
    new_vector[index] += increment
    return new_vector


def research(f, vector, delta_vector):
    v = vector
    deltas = delta_vector
    min_f = f(*v)
    success = False
    
    for i in range(len(deltas)):
        incremented_vector = increment_vector(v, i, deltas[i])
        f_incremented_value = f(*incremented_vector)
        
        if f_incremented_value < min_f:
            v = incremented_vector
            min_f = f_incremented_value
            success = True
        else:
            incremented_vector = increment_vector(v, i, -1 * deltas[i])
            f_incremented_value = f(*incremented_vector)
            if f_incremented_value < min_f:
                v = incremented_vector
                min_f = f_incremented_value
                success = True
                
    return v, min_f, success


def hook_jeeves_method(f, start_vector, delta_vector, increment, epsilon):
    vector = start_vector
    n = len(vector)
    delta = delta_vector
    f_value = f(*vector)
    
    while True:
        new_vector, new_min, success_research = research(f, vector, delta)
        if success_research:
            tmp_basis = []
            for i in range(n):
                tmp_basis.append(new_vector[i]*2 - vector[i])

            new_basis, new_basis_min, success = research(f, tmp_basis, delta)

            if new_basis_min < new_min:
                vector = new_basis
                f_value = new_basis_min
            else:
                vector = new_vector
                f_value = new_min
        else:
            if abs(vector_abs(delta)) < epsilon:
                break
            for i in range(n):
                delta[i] = delta[i] / increment

    return f_value, vector


target_function_min, vector_min = \
    hook_jeeves_method(
        hook_jeeves_target_function,
        hook_jeeves_x_start_vector,
        hook_jeeves_delta_x_start_vector,
        hook_jeeves_step_increment,
        hook_jeeves_accuracy)

x1_symbol, x2_symbol, f_min = symbols('x_1 x_2 f_min')

##### Min value for Hook-Jeeves' method task

In [198]:
relational.Eq(f_min, Float(str(target_function_min), 5))

fₘᵢₙ = 2.4414e-6

##### Min value coords for Hook-Jeeves' method task

In [199]:
relational.Eq(x1_symbol, Float(str(vector_min[0]), 5))

x₁ = 4.0016

In [200]:
relational.Eq(x2_symbol, Float(str(vector_min[1]), 5))

x₂ = 7.7716e-16

### Nelder-Mead's method

In [201]:
def sort_func(f):
    return lambda vector : f(*vector)


def find_mid(points):
    mid = points[0].copy()
    n = len(points) - 1
    for i in range(1, n):
        mid = vector_sum(points[i].copy(), mid)

    return multiply_vector(mid, 1 / n)


def multiply_vector(vector, number):
    res = vector.copy()
    for i in range(len(res)):
        res[i] *= number
    return res


def vector_sub(v1, v2):
    res = v1.copy()
    for i in range(len(res)):
        res[i] -= v2[i]
    return res


def vector_sum(v1, v2):
    res = v1.copy()
    for i in range(len(res)):
        res[i] += v2[i]
    return res

def check_convergence(f, points, epsilon):
    f_values = []
    for i in range(len(points)):
        f_values.append(f(*points[i]))
    f_std = std(f_values) # standard deviation
    return f_std < epsilon


def nelder_mead(f, start_points, epsilon, alpha=1, beta=0.5, gamma=2):
    points = start_points.copy()

    while True:
        points = sorted(points, key=sort_func(f))

        xh = points[len(points) - 1].copy()
        fh = f(*xh)
        xg = points[len(points) - 2].copy()
        fg = f(*xg)
        xl = points[0].copy()
        fl = f(*xl)

        mid = find_mid(points)

        xr = vector_sub(multiply_vector(mid, (1 + alpha)), multiply_vector(xh, alpha))
        fr = f(*xr)

        if fr < fl:
            xe = vector_sum(multiply_vector(xr, gamma), multiply_vector(mid, 1 - gamma))
            fe = f(*xe)
            xh = xe if fe < fl else xr
        elif fg > fr > fl:
            xh = xr
        elif fr > fg:
            if fr < fh:
                xh = xr
                fh = fr

            xc = vector_sum(multiply_vector(xh, beta), multiply_vector(mid, 1 - beta))
            fc = f(*xc)

            if fc < fh:
                xh = xc
            else:
                for j in range(len(points)):
                    points[j] = multiply_vector(vector_sum(xl, points[j]), 0.5)
                continue
        points[0] = xl
        points[len(points) - 1] = xh

        if check_convergence(f, points, epsilon):
            points = sorted(points, key=sort_func(f))
            break

    return points, f(*points[0])


final_simplex, nelder_min_f = nelder_mead(nelder_mead_target_function, nelder_mead_start, nelder_mead_accuracy)

##### Min point for Nelder-Mead

In [202]:
x_min = symbols('x_min')
relational.Eq(x_min, Array(final_simplex[0]))

xₘᵢₙ = [3.00661134719849  3.97772121429443]

##### Min value for Nelder-Mead

In [203]:
relational.Eq(f_min, Float(str(nelder_min_f), 5))

fₘᵢₙ = 0.00058376